In [1]:
import pandas as pd
import sqlite3


In [2]:
def load_input_files():
    orders_df = pd.read_csv("orders.csv")
    users_df = pd.read_json("users.json")
    return orders_df, users_df


In [3]:
def get_restaurant_data():
    connection = sqlite3.connect("restaurants.db")

    with open("restaurants.sql", "r", encoding="utf-8") as file:
        script = file.read()

    script = script.replace(
        "CREATE TABLE restaurants",
        "CREATE TABLE IF NOT EXISTS restaurants"
    )

    connection.executescript(script)

    rest_df = pd.read_sql_query("SELECT * FROM restaurants", connection)

    return rest_df


In [4]:
def build_master_dataset(o, u, r):
    step1 = o.merge(u, how="left", on="user_id")
    final_df = step1.merge(r, how="left", on="restaurant_id")
    return final_df


In [5]:
orders, users = load_input_files()
restaurants = get_restaurant_data()

data = build_master_dataset(orders, users, restaurants)

print("Final rows:", len(data))
data.head()


Final rows: 190000


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
2,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
3,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
4,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2


In [6]:
data.to_csv("final_dataset.csv", index=False)


In [7]:
print("Total Orders:", len(data))
print("Unique Users:", data["user_id"].nunique())
print("Cities:", data["city"].unique())


Total Orders: 190000
Unique Users: 2883
Cities: <StringArray>
['Hyderabad', 'Pune', 'Chennai', 'Bangalore']
Length: 4, dtype: str
